In [1]:
import numpy as np
import os
import json

In [2]:
from filtool_script import FiltoolExec

In [7]:
fil_exec = FiltoolExec('/home/rsenzel/Code/inception/dev_injector/fb/controlfb2_uhf.fil', 'mmgps_search.json', 'test_dir', 1)

In [8]:
fil_exec.fb

'/home/rsenzel/Code/inception/dev_injector/fb/controlfb2_uhf.fil dewf.fil'

In [4]:
fil_exec.processing_args

{'rfi_flags': 'kadaneF 8 4 zdot',
 'temp_filesystem': '/beegfs/',
 'fscrunch': 1,
 'snr_threshold': 8.0,
 'nharmonics': '4',
 'fft_length': 4194304,
 'start_accel': -105.0,
 'end_accel': 105.0,
 'accel_tol': 1.11,
 'candidate_limit': 1000,
 'ddplan': ' 0.000 103.800 0.03 1 3460 0.7544\r\n 103.800 167.300 0.05 2 1270 0.1384\r\n 167.300 318.500 0.10 4 1512 0.08242\r\n 318.500 500.100 0.20 8 908 0.02475',
 'overrides': 'both',
 'birdie_list': '1000.00:0.05,1500.00:0.05,1300.00:0.05',
 'channel_mask': '932.00:961.00',
 'ram_limit': 24.0}

In [2]:
import xml.etree.ElementTree as ET
import json

In [4]:
with open('/home/rsenzel/Code/inception/offline_injector_nextflow/xml_files/injection_report_inj_000001.json', 'r') as file:
    pars = json.load(file)

psr = pars['pulsars']

In [12]:
def xml_to_dict(element):
    # Convert element attributes and text to a dictionary
    data = {element.tag: {} if element.attrib else None}
    children = list(element)
    
    # Process each child
    if children:
        dd = {}
        for dc in map(xml_to_dict, children):
            for k, v in dc.items():
                if k in dd:
                    if not isinstance(dd[k], list):
                        dd[k] = [dd[k]]
                    dd[k].append(v)
                else:
                    dd[k] = v
        data = {element.tag: dd}
    if element.attrib:
        data[element.tag].update(('@' + k, v) for k, v in element.attrib.items())
    if element.text:
        text = element.text.strip()
        if children or element.attrib:
            if text:
                data[element.tag]['#text'] = text
        else:
            data[element.tag] = text
    return data

In [139]:
con1 = xml_to_dict(ET.parse('xml_files/control_1.xml').getroot())['peasoup_search']['candidates']['candidate']
con2 = xml_to_dict(ET.parse('xml_files/control_2.xml').getroot())['peasoup_search']['candidates']['candidate']
con3 = xml_to_dict(ET.parse('xml_files/control_3.xml').getroot())['peasoup_search']['candidates']['candidate']
con4 = xml_to_dict(ET.parse('xml_files/control_4.xml').getroot())['peasoup_search']['candidates']['candidate']

inj1 = xml_to_dict(ET.parse('xml_files/inj_1.xml').getroot())['peasoup_search']['candidates']['candidate']
inj2 = xml_to_dict(ET.parse('xml_files/inj_2.xml').getroot())['peasoup_search']['candidates']['candidate']
inj3 = xml_to_dict(ET.parse('xml_files/inj_3.xml').getroot())['peasoup_search']['candidates']['candidate']
inj4 = xml_to_dict(ET.parse('xml_files/inj_4.xml').getroot())['peasoup_search']['candidates']['candidate']

orig1 = xml_to_dict(ET.parse('xml_files/orig_1.xml').getroot())['peasoup_search']['candidates']['candidate']
orig2 = xml_to_dict(ET.parse('xml_files/orig_2.xml').getroot())['peasoup_search']['candidates']['candidate']
orig3 = xml_to_dict(ET.parse('xml_files/orig_3.xml').getroot())['peasoup_search']['candidates']['candidate']
orig4 = xml_to_dict(ET.parse('xml_files/orig_4.xml').getroot())['peasoup_search']['candidates']['candidate']

nm1 = xml_to_dict(ET.parse('xml_files/no_merge_1.xml').getroot())['peasoup_search']['candidates']['candidate']
nm2 = xml_to_dict(ET.parse('xml_files/no_merge_2.xml').getroot())['peasoup_search']['candidates']['candidate']
nm3 = xml_to_dict(ET.parse('xml_files/no_merge_3.xml').getroot())['peasoup_search']['candidates']['candidate']
nm4 = xml_to_dict(ET.parse('xml_files/no_merge_4.xml').getroot())['peasoup_search']['candidates']['candidate']

con = [*con1, *con2, *con3, *con4]
inj = [*inj1, *inj2, *inj3, *inj4]
orig = [*orig1, *orig2, *orig3, *orig4]
nm = [*nm1, *nm2, *nm3, *nm4]

len(con), len(nm), len(inj), len(orig)

(3232, 3232, 3226, 789)

In [136]:
per1 = np.array([float(dr['period']) for dr in con])
per2 = np.array([float(dr['period']) for dr in nm])

In [137]:
pd.Series(per1-per2).value_counts()

0.0    3232
Name: count, dtype: int64

In [138]:
inj_p0 = [psr_i['PX'][0] for psr_i in psr]
inj_dm = [psr_i['DM'] for psr_i in psr]
inj_snr = [round(psr_i['SNR'], 5) for psr_i in psr]
inj_bin = [psr_i['binary_period'] for psr_i in psr]
list(zip(inj_p0, inj_dm, inj_snr, inj_bin))

[(7.275772353384312, 358.0534015710773, 30.82365, None),
 (0.009728202344499039, 153.98138484575946, 17.63112, 7896.853214125578),
 (0.02269621631844739, 375.3319681941882, 31.94065, 12305.567298504631),
 (0.018373921349606668, 301.5548644157, 27.17122, 10836.123886101605),
 (0.018876105346978627, 310.1266257501524, 27.72536, 11006.85054221188),
 (0.013088825061494773, 211.3437381186177, 21.33939, 9039.358514819074),
 (0.0227152373094482, 375.6566368336848, 31.96164, 12312.033833094476),
 (0.002324015950151076, 27.599582597406297, 9.46098, 5379.664313365096),
 (0.0022246345544454245, 25.903244981051216, 9.35132, 5345.8777860022365),
 (0.0041639163274497185, 59.004778692676226, 11.49122, 6005.172176292015)]

In [119]:
def get_lens(cands):
    n_cands = []
    for i in range(len(inj_p0)):
        tol=0.05
        dm_tol = 0.05
        cand0 = []
        for ci in cands:
            cond1 = (float(ci['period']) > inj_p0[i]*(1-tol)) & (float(ci['period']) < inj_p0[i]*(1+tol))
            cond2 = (float(ci['dm']) > inj_dm[i]*(1-dm_tol)) & (float(ci['dm']) < inj_dm[i]*(1+dm_tol))
            if cond1 and cond2:
                cand0.append(ci)
        n_cands.append(len(cand0))
    return n_cands

In [120]:
get_lens(con), get_lens(nm), get_lens(inj), get_lens(orig), 

([0, 1, 1, 1, 1, 0, 1, 2, 0, 2],
 [0, 1, 0, 2, 2, 0, 0, 2, 0, 2],
 [0, 1, 3, 3, 3, 1, 3, 3, 0, 3],
 [0, 0, 0, 0, 0, 0, 0, 3, 0, 2])

In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [104]:
con_csv = pd.read_csv('xml_files/con_cands_to_fold.csv')
inj_csv = pd.read_csv('xml_files/inj_cands_to_fold.csv')
len(con_csv), len(inj_csv)

(23214, 23219)

In [24]:
import re

In [25]:
expr = "/tmp/rsenzel/MMGPS_U_0018_cfbf00125_data_000001_merged.fil"
match = re.search(r'[ci]fbf\d{5}', expr).group()
match

'cfbf00125'

In [26]:
def generate_pulsarX_cand_file(
        tmp_dir,
        beam_name,
        cand_mod_periods,
        cand_dms,
        cand_accs,
        cand_snrs):

    cand_file_path = f'{tmp_dir}/{beam_name}_cands.candfile' 
    with open(cand_file_path, 'w') as f:
        f.write("#id DM accel F0 F1 S/N\n")
        for i in range(len(cand_mod_periods)):
            f.write(
                "%d %f %f %f 0 %f\n" %
                (i,
                 cand_dms[i],
                    cand_accs[i],
                    1.0 /
                    cand_mod_periods[i],
                    cand_snrs[i]))
        f.close()

    return cand_file_path

In [13]:
tmp_dir="test_dir"
beam_name = match


'-i 258'

In [140]:
cmask = "856.0:863.94140625"
zap_string = ""
if cmask is not None:
    cmask = cmask.strip()
    if cmask:
        zap_string = " ".join(["--rfi zap {} {}".format(*i.split(":")) for i in cmask.split(",")])

In [142]:
cmask = cmask.strip()
cmask

'856.0:863.94140625'

In [105]:
con_csv = con_csv.reindex()
inj_csv = inj_csv.reindex()

In [107]:
locs = []
offset = 0
for i in range(len(con_csv)):
    con_i = con_csv.iloc[i+offset]
    inj_i = inj_csv.iloc[i]
    if (round(con_i['period'], 3) == round(inj_i['period'], 3)):
        pass
    else:
        locs.append(i)
        offset -= 1

In [116]:
# locs

In [109]:
i=8
inj_csv.iloc[i]['period'], con_csv.iloc[i]['period'], inj_csv.iloc[i]['dm'], con_csv.iloc[i]['dm']

(np.float64(0.0130741628977736),
 np.float64(2.00214078447418),
 np.float64(211.22900390625),
 np.float64(391.299987792969))

In [112]:
inj_csv.head(35)

,Unnamed: 0,level_0,index,period,opt_period,dm,acc,nh,snr,folded_snr,...,src_raj,src_rajd,src_dej,src_dejd,file_index,file,cluster_id,strongest_in_cluster,spatial_rfi,low_nassoc
0,0,0,23,1.002560,0,383.700012,-105.000000,0,27.812326,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,0,1,0,0
1,1,2,31,2.003133,0,404.500000,-105.000000,2,23.437431,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,1,1,0,0
2,2,3,32,2.005120,0,413.299988,-4.956955,1,23.043993,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,2,1,0,0
3,3,4,34,0.667490,0,268.100006,-105.000000,0,22.241913,0,...,121848.88,184.703667,-595533.5,-59.925972,1914,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,3,1,0,0
4,4,6,41,5.002874,0,404.500000,-105.000000,3,20.534908,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,4,1,0,0
5,5,7,50,1.427374,0,477.899994,-105.000000,0,18.481922,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,5,1,0,0
6,6,8,53,1.000575,0,198.199997,-105.000000,1,18.114378,0,...,121848.88,184.703667,-595533.5,-59.925972,1914,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,6,1,0,0
7,7,9,57,2.001150,0,271.700012,19.580593,2,17.628910,0,...,121848.88,184.703667,-595533.5,-59.925972,1914,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,7,1,0,0
8,8,10,62,0.013074,0,211.229004,-79.531494,1,17.233582,0,...,122053.45,185.222708,-594906.9,-59.818583,1918,/tmp/rsenzel/data_000001_inj_000001_overview_d...,8,1,0,0
9,9,11,65,2.002141,0,391.299988,105.000000,3,16.943661,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,9,1,0,0


In [113]:
con_csv.head(35)

,Unnamed: 0,level_0,index,period,opt_period,dm,acc,nh,snr,folded_snr,...,src_raj,src_rajd,src_dej,src_dejd,file_index,file,cluster_id,strongest_in_cluster,spatial_rfi,low_nassoc
0,0,0,23,1.002560,0,383.700012,-105.000000,0,27.812326,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,0,1,0,0
1,1,2,31,2.003133,0,404.500000,-105.000000,2,23.437431,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,1,1,0,0
2,2,3,32,2.005120,0,413.299988,-4.956955,1,23.043993,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,2,1,0,0
3,3,4,34,0.667490,0,268.100006,-105.000000,0,22.241913,0,...,121848.88,184.703667,-595533.5,-59.925972,1914,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,3,1,0,0
4,4,6,41,5.002874,0,404.500000,-105.000000,3,20.534908,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,4,1,0,0
5,5,7,50,1.427374,0,477.899994,-105.000000,0,18.481922,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,5,1,0,0
6,6,8,53,1.000575,0,198.199997,-105.000000,1,18.114378,0,...,121848.88,184.703667,-595533.5,-59.925972,1914,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,6,1,0,0
7,7,9,57,2.001150,0,271.700012,19.580593,2,17.628910,0,...,121848.88,184.703667,-595533.5,-59.925972,1914,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,7,1,0,0
8,8,10,64,2.002141,0,391.299988,105.000000,3,16.943661,0,...,121848.88,184.703667,-595533.5,-59.925972,1915,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,8,1,0,0
9,9,11,65,5.002874,0,245.100006,105.000000,3,16.851202,0,...,121848.88,184.703667,-595533.5,-59.925972,1914,/hercules/scratch/vishnu/MMGPS_UHF_DATA/MMGPS_...,9,1,0,0


In [ ]:
# 28	35	42	133	0.222203	0	96.209999	23.281223	0	12.724810

In [72]:
inj_csv.iloc[8] #0.013088825061494773, 211.3437381186177, 21.33939, 9039.358514819074

level_0                                                                10
index                                                                  62
period                                                           0.013074
opt_period                                                              0
dm                                                             211.229004
acc                                                            -79.531494
nh                                                                      1
snr                                                             17.233582
folded_snr                                                              0
is_adjacent                                                             1
is_physical                                                             1
ddm_count_ratio                                                  0.951923
ddm_snr_ratio                                                    0.966693
nassoc                                

In [13]:
ddplan = list(fil_exec.create_DDplan())
filplan = fil_exec.create_filplan(ddplan)

In [14]:
ddplan = fil_exec.create_DDplan()

In [26]:
np.unique(np.random.randint(0, 11, size=10000))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [30]:
arr = np.array(['abs', 'rty', 'fer', 'fre'])

with open("output.txt", "w") as file:
    # Add a newline character to each string and write to file
    file.writelines(line + "\n" for line in arr)

In [31]:
from scipy.stats import truncnorm

In [34]:
trvs = truncnorm(a=10, b=20, loc=25, scale=5).rvs

In [42]:
trvs(random_state=0)

np.float64(75.39255995729471)

In [43]:
2**23

8388608

In [11]:
f'overview_dm_0.000000_103.800000.xml'

'overview_dm_0.000000_103.800000.xml'

In [12]:
xml_file_names = [f'overview_dm_{dm_range.low_dm:.6f}_{dm_range.high_dm:.6f}.xml' for dm_range in ddplan]
xml_file_names

['overview_dm_0.000000_103.800000.xml',
 'overview_dm_103.800000_167.300000.xml',
 'overview_dm_167.300000_318.500000.xml',
 'overview_dm_318.500000_500.100000.xml']

In [6]:
for i in ddplan:
    print(i)

DMRange(low_dm=0.0, high_dm=517.8, dm_step=0.3, tscrunch=1.0)
DMRange(low_dm=517.8, high_dm=834.8, dm_step=0.5, tscrunch=2.0)
DMRange(low_dm=834.8, high_dm=1589.8, dm_step=1.0, tscrunch=4.0)
DMRange(low_dm=1589.8, high_dm=3001.8, dm_step=2.0, tscrunch=8.0)


In [45]:
dm_range = np.linspace(ddplan[0].low_dm, ddplan[0].high_dm, int((ddplan[0].high_dm-ddplan[0].low_dm)/ddplan[0].dm_step), endpoint=False)
dm1 = np.linspace(ddplan[1].low_dm, ddplan[1].high_dm, int((ddplan[1].high_dm-ddplan[1].low_dm)/ddplan[1].dm_step), endpoint=False)

In [82]:
dm_trials_list = []
tscrunch_list = []
for i, dm_range in enumerate(ddplan):
    n_trial = int((dm_range.high_dm-dm_range.low_dm)/dm_range.dm_step)
    n_trial, endpoint = (n_trial + 1, True) if i == len(ddplan)-1 else (n_trial, False)
    dm_list_i = np.linspace(dm_range.low_dm, dm_range.high_dm, n_trial, endpoint=endpoint)
    tscrunch = np.ones_like(dm_list_i) * dm_range.tscrunch
    dm_trials_list.append(dm_list_i)
    tscrunch_list.append(tscrunch)

In [83]:
dm_trials = np.concat(dm_trials_list)
tscrunch = np.concat(tscrunch_list)
tscrunch

array([1., 1., 1., ..., 8., 8., 8.])

In [66]:
def find_n_nearest(dm_arr, tscrunch_arr, target, n):
    nearest_indices = np.argsort(np.abs(dm_arr - target))[:n]
    dm_values = dm_arr[nearest_indices]
    tscrunch_values = tscrunch_arr[nearest_indices]
    return dm_values, tscrunch_values

In [71]:
a, b = find_n_nearest(dm_trials, tscrunch, 517.7, 5)
a, b

(array([517.8, 517.5, 517.2, 518.3, 516.9]), array([2., 1., 1., 2., 1.]))

In [81]:
np.where(np.array([dm_range.tscrunch for dm_range in ddplan] )== 2)[0][0]

np.int64(1)

In [73]:
a[np.where(b == 1)]

array([517.5, 517.2, 516.9])

In [50]:
my_dms = np.random.uniform(50, 1500, size=10)
nearest = 5
dm_search = np.zeros((len(my_dms), nearest))
my_dms, dm_search

(array([1072.13331736, 1374.67901752, 1121.25312234, 1487.44617915,
        1061.48080809, 1298.42985136, 1143.93419058,  585.98544057,
         226.25915701,  530.1386847 ]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))